# datasets.sentinel2

> Sentinel-2 presets for vegetation indices (NDVI, EVI) and spectral bands.

In [1]:
#| default_exp datasets.sentinel2

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import ee
from gee_polygons.layers import ContinuousLayer

/Users/alice/.local/share/mamba/envs/gee-polygons/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


## About Sentinel-2

Sentinel-2 is a European Space Agency (ESA) mission providing high-resolution optical imagery.

Key features:
- 10m resolution (visible and NIR bands)
- ~5 day revisit time
- 13 spectral bands
- Available from 2015 onwards

We use the **Surface Reflectance Harmonized** collection (`COPERNICUS/S2_SR_HARMONIZED`) which includes atmospheric correction.

## Vegetation Indices

The most common indices for monitoring vegetation health and recovery:

- **NDVI** (Normalized Difference Vegetation Index): `(NIR - Red) / (NIR + Red)`
- **EVI** (Enhanced Vegetation Index): `2.5 * (NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1)`

In [4]:
#| export
def add_indices(image):
    """Add NDVI and EVI bands to a Sentinel-2 image.

    Note: Expects raw Sentinel-2 SR reflectance (scaled by 10000).
    Scales to 0-1 range before computing indices.
    """
    # Scale reflectance from 0-10000 to 0-1 range
    scaled = image.divide(10000)

    ndvi = scaled.normalizedDifference(['B8', 'B4']).rename('NDVI')

    evi = scaled.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': scaled.select('B8'),
            'RED': scaled.select('B4'),
            'BLUE': scaled.select('B2')
        }
    ).rename('EVI')

    return image.addBands([ndvi, evi])


def mask_s2_clouds(image):
    """Mask clouds in Sentinel-2 imagery using QA60 band."""
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    
    mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0)
            .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))
    
    return image.updateMask(mask)

In [5]:
#| export
def get_s2_collection(start_date: str, end_date: str, geometry=None, cloud_pct: int = 20):
    """Get a processed Sentinel-2 collection with NDVI and EVI.
    
    Args:
        start_date: Start date (YYYY-MM-DD)
        end_date: End date (YYYY-MM-DD)
        geometry: Optional geometry to filter bounds
        cloud_pct: Maximum cloud cover percentage (default 20)
        
    Returns:
        ee.ImageCollection with NDVI and EVI bands added
    """
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_pct))
        .map(mask_s2_clouds)
        .map(add_indices)  # Scaling happens inside add_indices
    )
    
    if geometry is not None:
        collection = collection.filterBounds(geometry)
    
    return collection

## Presets

Note: NDVI and EVI are computed bands, so we need a slightly different approach.
The `ContinuousLayer` points to the raw collection, and the extraction handles index computation.

For now, we provide a simpler preset that assumes indices are pre-computed.

In [6]:
#| export
# These presets work with the raw Sentinel-2 collection
# The extract_continuous function should handle index computation

SENTINEL2_NDVI_EVI = ContinuousLayer(
    collection_id='COPERNICUS/S2_SR_HARMONIZED',
    bands=['NDVI', 'EVI'],
    scale=10,
    preprocess=add_indices
)

## Usage

```python
from gee_polygons.datasets.sentinel2 import SENTINEL2_NDVI, get_s2_collection

# Option 1: Use the preset with extract_continuous
df = site.extract_continuous(
    SENTINEL2_NDVI,
    start_date='2020-01-01',
    end_date='2024-12-31',
    reducer='mean',
    frequency='yearly'
)

# Option 2: Get the full collection for custom processing
collection = get_s2_collection('2020-01-01', '2024-12-31', site.geometry)
```

## Visualization Helpers

Functions for getting Sentinel-2 imagery ready for visualization with `gee_polygons.visualize`.

In [ ]:
#| export
from typing import Optional, Literal

# Band combinations
BANDS_TRUE_COLOR = ['B4', 'B3', 'B2']       # Red, Green, Blue
BANDS_FALSE_COLOR = ['B8', 'B4', 'B3']      # NIR, Red, Green (vegetation in red)
BANDS_AGRICULTURE = ['B11', 'B8', 'B2']     # SWIR, NIR, Blue

# Visualization presets (plain dicts for use with render_image)
SENTINEL_VIS = {'min': 0, 'max': 0.3, 'gamma': 1.4}
SENTINEL_VIS_BRIGHT = {'min': 0, 'max': 0.25, 'gamma': 1.6}

In [ ]:
#| export
def get_sentinel_composite(
    geometry: ee.Geometry,
    date_range: tuple[str, str],
    bands: list[str] = None,
    cloud_pct: int = 20,
    reducer: Literal['median', 'mean', 'mosaic'] = 'median'
) -> ee.Image:
    """Get a cloud-masked Sentinel-2 composite for visualization.
    
    This returns an ee.Image ready for use with `gee_polygons.visualize.render_image()`.
    
    Args:
        geometry: Region of interest
        date_range: (start_date, end_date) as 'YYYY-MM-DD' strings
        bands: Bands to include (default: true color B4/B3/B2)
        cloud_pct: Maximum cloud cover percentage for filtering
        reducer: Temporal reducer ('median', 'mean', 'mosaic')
        
    Returns:
        ee.Image with selected bands, scaled to 0-1 reflectance
        
    Example:
        from gee_polygons.datasets.sentinel2 import get_sentinel_composite, SENTINEL_VIS
        from gee_polygons.visualize import render_image
        
        image = get_sentinel_composite(
            geometry=site.geometry,
            date_range=('2020-06-01', '2020-08-31')
        )
        
        pil_img = render_image(
            image=image,
            region=site.geometry.buffer(500).bounds(),
            vis_params=SENTINEL_VIS,
            boundary=site.geometry
        )
    """
    if bands is None:
        bands = BANDS_TRUE_COLOR
    
    def preprocess(image):
        # Mask clouds
        qa = image.select('QA60')
        cloud_mask = 1 << 10
        cirrus_mask = 1 << 11
        mask = (qa.bitwiseAnd(cloud_mask).eq(0)
                .And(qa.bitwiseAnd(cirrus_mask).eq(0)))
        # Scale to 0-1
        return image.updateMask(mask).divide(10000)
    
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(date_range[0], date_range[1])
        .filterBounds(geometry)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_pct))
        .map(preprocess)
        .select(bands)
    )
    
    if reducer == 'mosaic':
        return collection.mosaic()
    else:
        reducer_fn = getattr(collection, reducer)
        return reducer_fn()

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()